<a href="https://colab.research.google.com/github/SanyamRawat/Group3-/blob/main/SMART_DAMAGELEAKAGE_DETECTION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
df = pd.read_csv("Gas_Sensors_Measurements.csv")

# Drop Serial Number as it's not a feature
df.drop(columns=["Serial Number"], inplace=True)

# Separate features and target variable
X = df.drop(columns=["Gas"])  # Sensor readings
y = df["Gas"]  # Target variable (Gas presence)

# Encode categorical target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Convert labels to numerical values

# Normalize sensor readings
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Convert model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save TFLite model
with open("gas_leak_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model training & conversion complete! Saved as 'gas_leak_model.tflite'.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5141 - loss: 1.2618 - val_accuracy: 0.7055 - val_loss: 0.7125
Epoch 2/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7480 - loss: 0.5732 - val_accuracy: 0.8523 - val_loss: 0.3791
Epoch 3/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8883 - loss: 0.3378 - val_accuracy: 0.9125 - val_loss: 0.2853
Epoch 4/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9226 - loss: 0.2565 - val_accuracy: 0.9016 - val_loss: 0.2367
Epoch 5/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9296 - loss: 0.2131 - val_accuracy: 0.9219 - val_loss: 0.2044
Epoch 6/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9353 - loss: 0.1787 - val_accuracy: 0.9320 - val_loss: 0.1857
Epoch 7/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9376 - loss: 0.1685 - val_accuracy: 0.9297 - val_loss: 0.1760
Epoch 8/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9390 - loss: 0.1548 - val_accuracy: 0

In [ ]:
import numpy as np

# Load trained TFLite model
tflite_model_path = "gas_leak_model.tflite"
with open(tflite_model_path, "rb") as f:
    model_data = f.read()

# Convert model to C array format
hex_array = ",".join(f"0x{b:02x}" for b in model_data)

c_code = f"""#include <cstdint>

const unsigned char gas_leakage_model[] = {{
  {hex_array}
}};
"""

# Save as header file
with open("gas_leakage_model.h", "w") as f:
    f.write(c_code)

print("gas_leakage_model.h has been generated successfully")

gas_leakage_model.h has been generated successfully
